In [1]:
#!pip install pyathena

In [2]:
from pyathena import connect
import pandas as pd
import yaml

S3_RESULTS_PATH='s3://ecmo-athena-query-grr/results/'
REGION='ap-southeast-2'

athena = connect(s3_staging_dir=S3_RESULTS_PATH, region_name=REGION)

In [3]:
# TODO ICD9 codes needed for each condition below
icd9_mappings = yaml.safe_load('''
    chroniccard_mhyn: 
        - 10000
    chronicpul_mhyn:
        - 10000
    asthma_mhyn:
        - 10000
    renal_mhyn:
        - 10000
    modliver_mhyn:
        - 10000
    mildliv_mhyn:
        - 10000
    chronicneu_mhyn: 
        - 10000
    malignantneo_mhyn:
        - 10000
    chronhaemo_mhyn:
        - 10000 
    aidshiv_mhyn:
        - 10000
    obesity_mhyn:   
        - 10000
    diabetiscomp_mhyn: 
        - 10000
    diabetes_mhyn:
        - 10000
    rheumatology_mhyr:
        - 10000
    dementia_mhyn:
        - 10000
    malnutrition_mhyn:
        - 10000
    smoker:
        - 3051
    former_smoker:
        - V1582
''')

icd9_mappings

{'chroniccard_mhyn': [10000],
 'chronicpul_mhyn': [10000],
 'asthma_mhyn': [10000],
 'renal_mhyn': [10000],
 'modliver_mhyn': [10000],
 'mildliv_mhyn': [10000],
 'chronicneu_mhyn': [10000],
 'malignantneo_mhyn': [10000],
 'chronhaemo_mhyn': [10000],
 'aidshiv_mhyn': [10000],
 'obesity_mhyn': [10000],
 'diabetiscomp_mhyn': [10000],
 'diabetes_mhyn': [10000],
 'rheumatology_mhyr': [10000],
 'dementia_mhyn': [10000],
 'malnutrition_mhyn': [10000],
 'smoker': [3051],
 'former_smoker': ['V1582']}

In [4]:
icd9_expressions = {column: ','.join([f"'{v}'" for v in values]) for column, values in icd9_mappings.items()}
icd9_expression = ',\n  '.join([f'bool_or(icd9_code IN ({expression})) AS {column}' for column, expression in icd9_expressions.items()])
mapped_icd9_expression = ', '.join([', '.join([f"'{v}'" for v in values]) for values in icd9_mappings.values()])
visible_column_names = ', '.join([c for c in icd9_mappings if c not in ['smoker', 'former_smoker']])

icd9_mapping_sql = f'''
icd9_mapping AS (
  SELECT 
  subject_id,
  {icd9_expression},
  bool_or(icd9_code NOT IN ({mapped_icd9_expression})) AS other_mhyn,
  filter(array_agg(icd9_code), x -> x NOT IN ({mapped_icd9_expression})) AS otherrisktext
  FROM diagnoses_icd
  GROUP BY subject_id
),
comorbidities AS (
  SELECT 
  subject_id, {visible_column_names}, other_mhyn,
  CASE
    WHEN smoker = true THEN 1
    WHEN former_smoker = true THEN 3
    ELSE 2
  END AS smoking_mhynfrom,
  otherrisktext      
  FROM icd9_mapping
)'''

'''icd9_unmapped AS (
    SELECT 
    subject_id,
    array_agg(icd9_code) AS otherrisktext,
    FROM diagnoses_icd
    WHERE icd9_code NOT IN ({mapped_icd9_expression})
    GROUP BY subject_id
)'''


comorbidities_sql = f'''WITH {icd9_mapping_sql}
SELECT * from comorbidities'''

print(comorbidities_sql)

WITH 
icd9_mapping AS (
  SELECT 
  subject_id,
  bool_or(icd9_code IN ('10000')) AS chroniccard_mhyn,
  bool_or(icd9_code IN ('10000')) AS chronicpul_mhyn,
  bool_or(icd9_code IN ('10000')) AS asthma_mhyn,
  bool_or(icd9_code IN ('10000')) AS renal_mhyn,
  bool_or(icd9_code IN ('10000')) AS modliver_mhyn,
  bool_or(icd9_code IN ('10000')) AS mildliv_mhyn,
  bool_or(icd9_code IN ('10000')) AS chronicneu_mhyn,
  bool_or(icd9_code IN ('10000')) AS malignantneo_mhyn,
  bool_or(icd9_code IN ('10000')) AS chronhaemo_mhyn,
  bool_or(icd9_code IN ('10000')) AS aidshiv_mhyn,
  bool_or(icd9_code IN ('10000')) AS obesity_mhyn,
  bool_or(icd9_code IN ('10000')) AS diabetiscomp_mhyn,
  bool_or(icd9_code IN ('10000')) AS diabetes_mhyn,
  bool_or(icd9_code IN ('10000')) AS rheumatology_mhyr,
  bool_or(icd9_code IN ('10000')) AS dementia_mhyn,
  bool_or(icd9_code IN ('10000')) AS malnutrition_mhyn,
  bool_or(icd9_code IN ('3051')) AS smoker,
  bool_or(icd9_code IN ('V1582')) AS former_smoker,
  bool_

In [6]:
df = pd.read_sql(comorbidities_sql, athena)

DatabaseError: Execution failed on sql: WITH 
icd9_mapping AS (
  SELECT 
  subject_id,
  bool_or(icd9_code IN ('10000')) AS chroniccard_mhyn,
  bool_or(icd9_code IN ('10000')) AS chronicpul_mhyn,
  bool_or(icd9_code IN ('10000')) AS asthma_mhyn,
  bool_or(icd9_code IN ('10000')) AS renal_mhyn,
  bool_or(icd9_code IN ('10000')) AS modliver_mhyn,
  bool_or(icd9_code IN ('10000')) AS mildliv_mhyn,
  bool_or(icd9_code IN ('10000')) AS chronicneu_mhyn,
  bool_or(icd9_code IN ('10000')) AS malignantneo_mhyn,
  bool_or(icd9_code IN ('10000')) AS chronhaemo_mhyn,
  bool_or(icd9_code IN ('10000')) AS aidshiv_mhyn,
  bool_or(icd9_code IN ('10000')) AS obesity_mhyn,
  bool_or(icd9_code IN ('10000')) AS diabetiscomp_mhyn,
  bool_or(icd9_code IN ('10000')) AS diabetes_mhyn,
  bool_or(icd9_code IN ('10000')) AS rheumatology_mhyr,
  bool_or(icd9_code IN ('10000')) AS dementia_mhyn,
  bool_or(icd9_code IN ('10000')) AS malnutrition_mhyn,
  bool_or(icd9_code IN ('3051')) AS smoker,
  bool_or(icd9_code